#### 케라스에서 MNIST 데이터셋 적재하기

* 머신 러닝에서 분류 문제의 범주 **(category)** 를 **class**
* 데이터 포인트는 **sample**
* 특정 샘플의 클래스는 **label** 

In [1]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [2]:
train_images.shape

(60000, 28, 28)

In [3]:
len(train_labels)

60000

In [4]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [5]:
test_images.shape

(10000, 28, 28)

In [6]:
len(test_labels)

10000

In [7]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

#### 신경망 구조

작업 순서는:

1. 훈련 데이터 `train_images`와 `train_labels`를 네트워크에 주입
2. 네트워크는 이미지와 레이블을 연관시킬 수 있도록 학습
3. test_images에 대한 예측을 요청

In [8]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(28*28, activation = "relu", input_shape=(28 * 28, )))
network.add(layers.Dense(10, activation = "softmax"))

신경망의 핵심 구성 요소는 일종의 데이터 처리 필터라고 생각할 수 있는 **layer**

* 어떤 데이터가 들어가면 더 유용한 형태로 출력됨
* 구체적으로 층은 주어진 문제에 더 의미있는 표현 **(representation)** 을 입력된 데이터로 부터 추출

이 예시에서는 **완전 연결된 (fully connected)** 신경망 층인 Dense 층 2개가 연속되어있음

* 두번째 층은 10개의 확률 점수가 들어있는 배열을 반환하는 **softmax**층

#### 컴파일 단계

신경망이 훈련 준비를 마치기 위해서 컴파일 단계에 포함될 3가지가 더 필요

* 손실 함수 (**loss function**): 훈련 데이터에서 신경망의 성능을 측정하는 방법으로 네트쿼크가 옳은 방향으로 학습될 수 있도록 도와줌
* 옵티마이저 (**optimizer**): 입력된 데이터와 손실 함수를 기반으로 네트워크를 업데이트하는 메커니즘
* 모니터링 지표

In [9]:
network.compile(optimizer = "rmsprop",
               loss = "categorical_crossentropy",
               metrics = ["accuracy"])

#### 이미지 데이터 준비하기

훈련을 시작하기 전에 데이터를 네트쿼으케 맞는 크기로 바꾸고 모든 값을 0과 1 사이로 스케일을 조정함

* 신경망은 입력 데이터의 스케일에 민감하여 적절한 데이터 전처리 과정이 필요함.
* 이미지의 경우 보통 픽셀의 최댓값인 255로 나누어 사용

예를 들면 이 예시에서는 [0, 255] 사이의 값인 uint8 타입의 (60000, 28, 28) 크기를 가진 배열로 저장되어 있음.

* 이 데이터를 0과 1 사이의 값을 가지는 float32 타입의 (60000, 28 * 28) 크기의 배열로 바꿔줌

In [10]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32")/255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32")/255

#### 레이블 준비하기

In [11]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

#### 모델 학습

In [12]:
network.fit(train_images, train_labels, epochs = 5, batch_size = 128)

Epoch 1/5
469/469 [==============================] - 1s 1ms/step - loss: 0.4060 - accuracy: 0.8784
Epoch 2/5
469/469 [==============================] - 1s 1ms/step - loss: 0.0994 - accuracy: 0.9709
Epoch 3/5
469/469 [==============================] - 1s 1ms/step - loss: 0.0601 - accuracy: 0.9817
Epoch 4/5
469/469 [==============================] - 1s 1ms/step - loss: 0.0422 - accuracy: 0.9876
Epoch 5/5
469/469 [==============================] - 1s 1ms/step - loss: 0.0304 - accuracy: 0.9908


훈련하는 동안 2개의 정보가 출력됨

* 훈련 데이터에 대한 네트워크의 손실 **loss**와 정확도 **acc**

훈련 데이터에 대해 0.9909 (99.09%)의 정확도를 달성.

* 이 모델이 테스트 데이터에도 잘 작동하는지 확인

In [13]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 [==============================] - 0s 924us/step - loss: 0.0645 - accuracy: 0.9810


In [14]:
print("test_acc:", test_acc)

test_acc: 0.9810000061988831


테스트 세트의 정확도는 98.22%로 훈련 세트 정확도보다 낮음.

* 이 차이는 과대적합 **overfitting**때문
* 이는 머신 러닝 모델이 훈련 데이터보다 새로운 데이터에서 성능이 낮아지는 경향을 말함

#### Put it all together

In [15]:
from keras import models
from keras import layers
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32")/255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32")/255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

network = models.Sequential()
network.add(layers.Dense(28*28, activation = "relu", input_shape=(28 * 28, )))
network.add(layers.Dense(10, activation = "softmax"))

network.compile(optimizer = "rmsprop",
               loss = "categorical_crossentropy",
               metrics = ["accuracy"])

network.fit(train_images, train_labels, epochs = 5, batch_size = 128)

network.evaluate(test_images, test_labels)

Epoch 1/5
469/469 [==============================] - 1s 1ms/step - loss: 0.4092 - accuracy: 0.8780
Epoch 2/5
469/469 [==============================] - 1s 1ms/step - loss: 0.1000 - accuracy: 0.9705
Epoch 3/5
469/469 [==============================] - 1s 1ms/step - loss: 0.0587 - accuracy: 0.9820
Epoch 4/5
469/469 [==============================] - 1s 1ms/step - loss: 0.0415 - accuracy: 0.9871
Epoch 5/5
313/313 [==============================] - 0s 952us/step - loss: 0.0629 - accuracy: 0.9820


[0.06285513937473297, 0.9819999933242798]